<a href="https://colab.research.google.com/github/unmo/nlp_for_bert/blob/main/memo/session5/sesection5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語ニュースの分類
* liverdoorコーパスを利用する

In [1]:
!pip install transformers
!pip install datasets
!pip install nlp
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 32.2 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
     |████████████████████████████████| 61 kB 499 kB/s 
     |████████████████████████████████| 895 kB 33.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 243 kB 34.6 MB/s 
     |████████████████████████████████| 132 kB 46.8 MB/s 
     |████████████████████████████████| 1.1 MB 33.5 MB/s 
     |████████████████████████████████| 160 kB 39.2 MB/s 
     |████████████████████████████████| 271 kB 39.9 MB/s 
     |████████████████████████████████| 192 kB 38.6 MB/s 
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 490 kB 5.3 MB/s 
     |████████████████████████████████| 13.4 

## google driveのマウント

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## データセットの読み込み

In [24]:
from pathlib import Path

path = Path("/content/drive/MyDrive/NLP/livedoor")
dirs = [f for f in path.iterdir() if f.is_dir()]

dir_count = 0
file_count = 0
text_label_data = []

for idx, dir in enumerate(dirs):
    print(dir)
    dir_count += 1
    files = dir.glob("*.txt")
    for file in files:
        if file.name == "LICENSE.txt":
            continue

        with open(file, "r") as f:
            print(f"filename: {file.name}")
            text = f.readlines()[3:]
            text = "".join(text)
            text = text.translate(str.maketrans({"\n": "", "\t": "", "\r": "", "\u3000": ""}))
            text_label_data.append([text, idx])

        file_count += 1
        print(f"files: {file_count}, dirs: {dir_count}")


        




ストリーミング出力は最後の 5000 行に切り捨てられました。
filename: smax-6704860.txt
files: 4869, dirs: 7
filename: smax-6701425.txt
files: 4870, dirs: 7
filename: smax-6624204.txt
files: 4871, dirs: 7
filename: smax-6557974.txt
files: 4872, dirs: 7
filename: smax-6565906.txt
files: 4873, dirs: 7
filename: smax-6805310.txt
files: 4874, dirs: 7
filename: smax-6606033.txt
files: 4875, dirs: 7
filename: smax-6558071.txt
files: 4876, dirs: 7
filename: smax-6581498.txt
files: 4877, dirs: 7
filename: smax-6526297.txt
files: 4878, dirs: 7
filename: smax-6855347.txt
files: 4879, dirs: 7
filename: smax-6725137.txt
files: 4880, dirs: 7
filename: smax-6594409.txt
files: 4881, dirs: 7
filename: smax-6535933.txt
files: 4882, dirs: 7
filename: smax-6538588.txt
files: 4883, dirs: 7
filename: smax-6557600.txt
files: 4884, dirs: 7
filename: smax-6749245.txt
files: 4885, dirs: 7
filename: smax-6563588.txt
files: 4886, dirs: 7
filename: smax-6742773.txt
files: 4887, dirs: 7
filename: smax-6869226.txt
files: 4888, dirs: 7
filename

## データの準備

In [51]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

train, test = train_test_split(text_label_data, shuffle=True)
output_path = Path("/content/drive/MyDrive/NLP/livedoor/csv")

if not output_path.exists():
    output_path.mkdir()

train_df = pd.DataFrame(train, columns=["text", "label"])
test_df = pd.DataFrame(test, columns=["text", "label"])

if not Path(f"{output_path}/train.csv").exists:
    train_df.to_csv(f"{output_path}/train.csv")

if not Path(f"{output_path}/test.csv").exists:
    test_df.to_csv(f"{output_path}/test.csv")

## モデルとTokenizerの読み込み

In [64]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

model_path = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=9)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_path)
model.cuda()
print(model.config)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.14.1",

## データセットの読み込み

In [73]:
from datasets import load_dataset

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, max_length=128, truncation=True)

input_path = "/content/drive/MyDrive/NLP/livedoor/csv"
train_data = load_dataset("csv", data_files=f"{input_path}/train.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

test_data = load_dataset("csv", data_files=f"{input_path}/test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

Using custom data configuration default-fa44822b338c78a0
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-fa44822b338c78a0/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-fa44822b338c78a0/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-e8772916963ccbc5.arrow
Using custom data configuration default-eb0acf3df1e359f6
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-eb0acf3df1e359f6/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb0acf3df1e359f6/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-2638cbcc36ad7617.arrow
